# Laboratorio 1 - Práctica 2.
### Gonzalo de las Heras de Matías - Jorge de la Fuente Tagarro
## Patrones secuenciales

![title](Images/img.jpg)

### Objetivo 

La biblioteca de la Universidad está pensando en reordenarla de manera que los usuarios pierdan el menos
tiempo posible buscando lo que necesitan. Basándose en el orden de consulta de varias fuentes, pretende que
la información esté lo más próxima posible entre ella. Para ello disponemos del Dataset library.csv que tiene
información del orden de consulta que ha realizado cada usuario. En cada registro tendremos el id del usuario,
el momento en que empezó a consultar la información (en milisegundos) y las fuentes consultadas.
Implementar el algoritmo GSP para obtener patrones secuenciales. 

#### Librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import re
from matplotlib.pyplot import cm
from AprioriSeq import APrioriSeq
from IPython.display import clear_output
warnings.filterwarnings('ignore')

#### Funciones

In [2]:
def Parser(Cadena):
    caracteres_eliminar = [
           "a","b","c","d","e","f","g","h","i","k",
           "l","m","n","o","p","q","r","s","t","u",
           "v","w","x","y","z", "-", "_", "0", "1",
           "2", "3", "4", "5", "6", "7", "8", "9",".", "%"]
    array = re.split(';|\t', Cadena) 
    row = np.array(array[3:])
    row_d = np.delete(row, np.where(row == "NULL"), axis=0)
    for caracter in caracteres_eliminar:
        row_d = np.delete(row_d, np.where(row_d == caracter), axis=0)
        row_d = np.unique(row_d, axis=0)
    return np.array([array[0], array[1], array[2], np.array(row_d)], dtype=object)

<font color="blue">Antes de ejecutar el algoritmo apriori con el dataset de la biblioteca, probaremos la librería con 1 dataset de prueba estableciendo una frecuencia de soporte mínima para comprobar el correcto funcionamiento de la librería.</font>

## Dataset dummy
<br>
<font color="blue">Establecemos una frecuencia soporte mínima de 2.</font>

In [3]:
apriori = APrioriSeq()
apriori.ExcelPrueba("Datos/dummy.xlsx")
apriori.CalcularReglasDeConfianza(FreSoporteMin=2, Echo=False)


Iniciando algoritmo apriori para patrones de asociación...

Itemset:



,0
0,A
1,B
2,C
3,D
4,E
5,F
6,G



-------

Probando k = 1
Calculando combinaciones
Calculando Frec.Soporte
Calculando Soporte


,Item,Frec. Soporte,Soporte
0,[A],4,0.8
1,[B],5,1.0
2,[C],3,0.6
3,[D],4,0.8
5,[F],4,0.8
6,[G],4,0.8



-------

Probando k = 2
Calculando combinaciones
Calculando Frec.Soporte
Calculando Soporte
Filtrando Soporte mínimo >= 2


...


-------

Probando k = 3
Calculando combinaciones
Calculando Frec.Soporte
Calculando Soporte
Filtrando Soporte mínimo >= 2


,Item,Frec. Soporte,Soporte
0,"[A, B, D]",2,0.4
1,"[A, B, F]",2,0.4
2,"[A, B, G]",2,0.4
3,"[A, C, D]",3,0.6
4,"[A, F, D]",2,0.4
5,"[A, G, D]",2,0.4
6,"[B, C, D]",2,0.4
7,"[B, F, D]",2,0.4
8,"[B, G, D]",3,0.6
9,"[F, AB]",2,0.4



-------

Probando k = 4
Calculando combinaciones
Calculando Frec.Soporte
Calculando Soporte
Filtrando Soporte mínimo >= 2


,Item,Frec. Soporte,Soporte
0,"[A, B, F, D]",2,0.4
1,"[A, B, G, D]",2,0.4



Terminado


<br>
<font color="blue">Las secuencias candidatas que cumplen la frecuencia soporte mínima son A, B, F, D y A, B, G, D.</font>

## Dataset librería
<br>
<font color="blue">Primero tenemos que preparar el dataset para que sea admitido por la librería de AprioriSeq.</font>

In [4]:
Fichero = "Datos/Datos.json"
try:
    Dataframe = pd.read_json(Fichero)
    print("Fichero cargado con éxito.")
except:
    print("No existe el fichero: " + Fichero)
    print("Creando fichero de datos...")
    Dataframe = pd.read_csv("Datos/library.csv")
    Datos = pd.DataFrame(columns=["id", "timestamp", "seconds", "sources"])
    for index, row in Dataframe.iterrows():
        clear_output(wait=True)
        print(index)
        Datos.loc[index] = Parser(Dataframe.iloc[index,0])
    Datos.to_json(Fichero)
    print("Fichero creado con éxito.")
    Dataframe = pd.read_json(Fichero)
    print("Fichero cargado con éxito.")
Dataframe = Dataframe[Dataframe["id"] != "NULL"]
Dataframe = Dataframe.groupby('id')['sources'].apply(list)
Dataframe = Dataframe.reset_index(drop=False)
del Dataframe['id']
print("Hay " + str(len(Dataframe)) + " registros.")

Fichero cargado con éxito.
Hay 14244 registros.


<font color="blue">Ya tenemos preparado el dataset para ser procesado por la librería de AprioriSeq.</font>

In [5]:
apriori = APrioriSeq()
apriori.CargaDataframe(Datos=Dataframe, Columna="sources")
apriori.CalcularReglasDeConfianza(FreSoporteMin=200, Echo=False)

,sources,items
0,"[[Arithmetic, Fundamental_theorem_of_arithmeti...","[Arithmetic, Fundamental_theorem_of_arithmetic..."
1,"[[Animal, Cambrian, Dinosaur, Fossil, Scientif...","[[Africa, Erwin_Rommel, North_Africa, Zimbabwe..."
2,"[[Blitzkrieg, Canada, Cold_War, NATO, Nazi_Ger...","[Blitzkrieg, Canada, Cold_War, NATO, Nazi_Germ..."
3,"[[Animal, Biology, Chemistry, Ecology, Extinct...","[Animal, Biology, Chemistry, Ecology, Extincti..."
4,"[[Ancient_Greece, Europe, Geography, Malta, So...","[[Ancient_Greece, Europe, Geography, Malta, So..."
5,"[[Bob_Dylan, Europe, Germany, John_F._Kennedy,...","[[Asteroid, British_Isles, Bronze_Age, Iron_Ag..."
6,"[[Chocolate, Computer, Dog, Industry, Telephone]]","[Chocolate, Computer, Dog, Industry, Telephone]"
7,"[[Akkadian_Empire, Babylonia, Cambodia, Hammur...","[Akkadian_Empire, Babylonia, Cambodia, Hammura..."
8,"[[Batman, Sun, Superman, Vitamin_D], [Baghdad,...","[[Adolf_Hitler, Africa, American_Civil_War, An..."
9,"[[Eye, Grand_Central_Terminal, Human, New_York...","[Africa, Asia, Eye, Grand_Central_Terminal, Ho..."



Iniciando algoritmo apriori para patrones de asociación...

Itemset:



,0
0,%C3%85land
1,%C3%89douard_Manet
2,%C3%89ire
3,%E2%82%AC2_commemorative_coins
4,10th_century
5,11th_century
6,12th_century
7,13th_century
8,14th_century
9,15th_Marine_Expeditionary_Unit



-------

Probando k = 1
Calculando combinaciones
Calculando Frec.Soporte
Calculando Soporte


,Item,Frec. Soporte,Soporte
17,[18th_century],201,0.014111
21,[19th_century],419,0.029416
34,[20th_century],581,0.040789
60,[AIDS],215,0.015094
110,[Adolf_Hitler],595,0.041772
115,[Africa],2083,0.146237
134,[Agriculture],872,0.061219
151,[Albert_Einstein],264,0.018534
197,[American_Civil_War],228,0.016007
217,[Ancient_Egypt],397,0.027871



-------

Probando k = 2
Calculando combinaciones
Calculando Frec.Soporte
Calculando Soporte
Filtrando Soporte mínimo >= 200


,Item,Frec. Soporte,Soporte
0,"[19th_century, United_States]",224,0.015726
1,"[20th_century, United_States]",294,0.020640
2,"[Adolf_Hitler, Germany]",202,0.014181
3,"[Adolf_Hitler, United_States]",257,0.018043
4,"[Adolf_Hitler, World_War_II]",201,0.014111
5,"[Africa, Africa]",455,0.031943
6,"[Africa, Agriculture]",241,0.016919
7,"[Africa, Animal]",289,0.020289
8,"[Africa, Asia]",250,0.017551
9,"[Africa, Asteroid]",232,0.016288



-------

Probando k = 3
Calculando combinaciones
Calculando Frec.Soporte
Calculando Soporte
Filtrando Soporte mínimo >= 200


,Item,Frec. Soporte,Soporte
0,"[Africa, Africa, United_States]",251,0.017621
1,"[Africa, Earth, United_States]",252,0.017692
2,"[Africa, England, United_States]",246,0.017270
3,"[Africa, Europe, Europe]",236,0.016568
4,"[Africa, Europe, United_Kingdom]",236,0.016568
5,"[Africa, Europe, United_States]",305,0.021413
6,"[Africa, North_America, United_States]",200,0.014041
7,"[Africa, United_Kingdom, United_Kingdom]",204,0.014322
8,"[Africa, United_Kingdom, United_States]",267,0.018745
9,"[Africa, United_States, Earth]",205,0.014392



Terminado


<font color="blue">Tras varias ejecuciones del algoritmo con distintos valores mínimos de frecuencia soporte, escogemos 200. Esto se debe a que si es más bajo, las secuencias candidatas obtenidas son muy largas, conllevando mucho tiempo de ejecución y las secuencias no dan mucha información. En cambio, si es más alto, obtenemos muy pocas secuencias candidatas que son muy cortas y dan poca información. 
<br><br>
Observando las secuencias candidatas nos damos cuenta que los patrones secuenciales de búsquedas tienen que ver con países de la segunda guerra mundial.
</font>